In [26]:
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import vgg16
from keras import backend as K
from keras.preprocessing import image
from keras.models import Sequential
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))
import tensorboard

In [27]:
input_tensor = Input(shape=(224,224,3))
model = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [28]:
model

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [30]:
for layer in model.layers:
    
    layer.trainable = False


In [31]:
model_ft = Sequential()

for layer in model.layers:
    model_ft.add(layer)


In [32]:
model_ft.add(Flatten())
model_ft.add(Dense(1024, activation='relu'))
#model_ft.add(Dropout(0.3))
model_ft.add(Dense(1024, activation='relu'))
#model_ft.add(Dropout(0.3))
model_ft.add(Dense(29, activation='softmax'))

In [33]:
model_ft.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [34]:
import keras

sgd_apna =keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0001, amsgrad=False)
model_ft.compile(optimizer=sgd_apna, loss='categorical_crossentropy', metrics=['accuracy'])


In [35]:
# import pickle

In [36]:
# # ValData= []
# with open("ValData.pkl","rb") as f:
#     ValData = pickle.load(f)

In [37]:
# TrainData= []
# with open("TrainData.pkl","rb") as f:
#     TrainData = pickle.load(f)

In [38]:
# len(TrainData)

In [39]:
# imgData=[]
# imgLabels=[]
# validityData = []
# validityLabels = []
# for i in range(len(TrainData)):
#     imgData.append(TrainData[i][0])
#     imgLabddels.append(TrainData[i][1])
    
# for i in range(len(ValData)):
#     validityData.append(ValData[i][0])
#     validityLabels.append(ValData[i][1])


In [40]:
# def getImages(img_path):
#     img = image.load_img(img_path, target_size=(224, 224))
#     x = image.img_to_array(img)
#     return x

In [41]:
# finalTrainData= []
# count = 0
# for i in range(len(imgData)):
#     count += 1
#     print(count)
#     #print(imgData[i])
#     finalTrainData.append(getImages(imgData[i]))

In [42]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
batch_size = 64

#train_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator(
        #rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        #horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(224, 224),  
        batch_size=batch_size,
        class_mode='categorical'
) 


Found 69600 images belonging to 29 classes.


In [43]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [44]:
validation_generator = val_datagen.flow_from_directory(
        'data/validation',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

Found 17400 images belonging to 29 classes.


In [45]:
callbacks = [keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)]

history = model_ft.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,callbacks=callbacks
      )
 
# Save the model
model_ft.save('small_last4.h5')

Epoch 1/30
1087/1087 [==============================] - 579s 533ms/step - loss: 2.0028 - acc: 0.5592 - val_loss: 0.6420 - val_acc: 0.8060
Epoch 2/30
1087/1087 [==============================] - 576s 530ms/step - loss: 0.4985 - acc: 0.8490 - val_loss: 0.3060 - val_acc: 0.8996
Epoch 3/30
1087/1087 [==============================] - 579s 533ms/step - loss: 0.2884 - acc: 0.9107 - val_loss: 0.1995 - val_acc: 0.9351
Epoch 4/30
1087/1087 [==============================] - 574s 528ms/step - loss: 0.1974 - acc: 0.9376 - val_loss: 0.1654 - val_acc: 0.9458
Epoch 5/30
1087/1087 [==============================] - 581s 535ms/step - loss: 0.1497 - acc: 0.9526 - val_loss: 0.1023 - val_acc: 0.9678
Epoch 6/30
1087/1087 [==============================] - 586s 539ms/step - loss: 0.1173 - acc: 0.9630 - val_loss: 0.0859 - val_acc: 0.9706
Epoch 7/30
1087/1087 [==============================] - 587s 540ms/step - loss: 0.1017 - acc: 0.9674 - val_loss: 0.0772 - val_acc: 0.9745
Epoch 8/30
1087/1087 [============